In [1]:
import pandas as pd

df = pd.read_pickle('../resources/data/preprocessed/tournament_games_without_ordinals.pkl')

df

,winner,season,day_number,team0,team1,prev_matchups,common_opps_win_pct_diff,elo_diff,ordinal_elo_diff
0,0,2003,134,1421,1411,NaN,NaN,-31.631037,17
1,0,2003,136,1112,1436,NaN,-0.333333,-469.244650,133
2,0,2003,136,1113,1272,NaN,NaN,120.473581,-31
3,0,2003,136,1141,1166,NaN,0.000000,117.657793,-22
4,0,2003,136,1143,1301,NaN,NaN,-54.718516,22
...,...,...,...,...,...,...,...,...,...
2491,1,2022,146,1274,1242,NaN,0.000000,239.449424,-45
2492,1,2022,146,1389,1314,NaN,NaN,212.406858,-88
2493,1,2022,152,1437,1242,NaN,-0.083333,7.104325,-1
2494,1,2022,152,1181,1314,0.5,-0.094203,-85.575138,11


In [2]:
df_rankings = pd.read_csv('../resources/data/unprocessed/MMasseyOrdinals.csv')

df_rankings

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,35,SEL,1102,159
1,2003,35,SEL,1103,229
2,2003,35,SEL,1104,12
3,2003,35,SEL,1105,314
4,2003,35,SEL,1106,260
...,...,...,...,...,...
4922507,2023,128,WOL,1473,332
4922508,2023,128,WOL,1474,166
4922509,2023,128,WOL,1475,260
4922510,2023,128,WOL,1476,301


In [3]:
df_rankings_2023 = df_rankings.loc[df_rankings['Season'] == 2023, :].reset_index(drop=True).copy()

df_rankings_2023 = pd.concat([df_rankings_2023, 
                              pd.read_csv('../resources/data/unprocessed/MMasseyOrdinals_2023_133.csv')], 
                             ignore_index=True)

df_rankings_2023

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2023,16,AP,1104,18
1,2023,16,AP,1112,14
2,2023,16,AP,1116,9
3,2023,16,AP,1120,13
4,2023,16,AP,1124,5
...,...,...,...,...,...
319183,2023,133,WOL,1473,333
319184,2023,133,WOL,1474,167
319185,2023,133,WOL,1475,258
319186,2023,133,WOL,1476,304


In [4]:
last_available = df_rankings_2023.sort_values(
    ['RankingDayNum']
).groupby(['SystemName']).tail(1).reset_index(drop=True)[['SystemName', 'RankingDayNum']]

last_available

,SystemName,RankingDayNum
0,RME,128
1,SMS,128
2,SEL,128
3,DAV,128
4,ARG,128
...,...,...
62,KCX,133
63,EBP,133
64,INC,133
65,JJK,133


In [5]:
df_rankings = df_rankings.loc[df_rankings['Season'] != 2023, :]

df_rankings = df_rankings.merge(last_available, how='inner', on=['SystemName', 'RankingDayNum'])

df_rankings

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank
0,2003,133,BIH,1102,172
1,2003,133,BIH,1103,177
2,2003,133,BIH,1104,40
3,2003,133,BIH,1105,312
4,2003,133,BIH,1106,269
...,...,...,...,...,...
197676,2022,133,OMN,1468,200
197677,2022,133,OMN,1469,284
197678,2022,133,OMN,1470,237
197679,2022,133,OMN,1471,254


In [6]:
df_rankings = df_rankings.pivot(index=['Season', 'TeamID'], columns=['SystemName'], 
                                values=['OrdinalRank']).reset_index()
df_rankings.columns = [i[1] if i[1] else i[0] for i in df_rankings.columns]

df_rankings

,Season,TeamID,7OT,AP,ARG,ATP,BAR,BBT,BIH,BNZ,...,STH,TPR,TRK,TRP,USA,WIL,WLK,WMV,WOB,WOL
0,2003,1102,NaN,NaN,NaN,NaN,NaN,NaN,172.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,165.0,NaN,155.0,157.0
1,2003,1103,NaN,NaN,NaN,NaN,NaN,NaN,177.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,172.0,NaN,177.0,171.0
2,2003,1104,NaN,NaN,NaN,NaN,NaN,NaN,40.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,36.0,NaN,37.0,37.0
3,2003,1105,NaN,NaN,NaN,NaN,NaN,NaN,312.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,310.0,NaN,312.0,311.0
4,2003,1106,NaN,NaN,NaN,NaN,NaN,NaN,269.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,254.0,NaN,265.0,270.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6886,2022,1468,158.0,NaN,148.0,206.0,223.0,196.0,155.0,202.0,...,189.0,215.0,176.0,196.0,NaN,164.0,181.0,186.0,148.0,155.0
6887,2022,1469,234.0,NaN,254.0,327.0,286.0,277.0,249.0,278.0,...,257.0,301.0,275.0,290.0,NaN,230.0,277.0,260.0,237.0,251.0
6888,2022,1470,206.0,NaN,227.0,226.0,216.0,227.0,226.0,201.0,...,203.0,183.0,203.0,236.0,NaN,215.0,231.0,215.0,224.0,225.0
6889,2022,1471,267.0,NaN,260.0,273.0,260.0,248.0,252.0,263.0,...,240.0,267.0,269.0,275.0,NaN,244.0,270.0,257.0,256.0,256.0


In [7]:
df_rankings = df_rankings.loc[:, df_rankings.isnull().mean() < 0.4]

df_rankings

,Season,TeamID,BIH,COL,DCI,DOK,DOL,DUN,KPK,KRA,...,PGH,POM,REW,RTH,SAG,STH,WIL,WLK,WOB,WOL
0,2003,1102,172.0,162.0,NaN,NaN,175.0,97.0,NaN,NaN,...,NaN,160.0,NaN,146.0,149.0,NaN,NaN,165.0,155.0,157.0
1,2003,1103,177.0,172.0,NaN,NaN,174.0,165.0,NaN,NaN,...,NaN,163.0,NaN,168.0,172.0,NaN,NaN,172.0,177.0,171.0
2,2003,1104,40.0,43.0,NaN,NaN,39.0,43.0,NaN,NaN,...,NaN,33.0,NaN,31.0,37.0,NaN,NaN,36.0,37.0,37.0
3,2003,1105,312.0,310.0,NaN,NaN,315.0,306.0,NaN,NaN,...,NaN,307.0,NaN,312.0,312.0,NaN,NaN,310.0,312.0,311.0
4,2003,1106,269.0,256.0,NaN,NaN,266.0,305.0,NaN,NaN,...,NaN,263.0,NaN,274.0,268.0,NaN,NaN,254.0,265.0,270.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6886,2022,1468,155.0,154.0,195.0,201.0,150.0,193.0,150.0,173.0,...,157.0,199.0,163.0,186.0,195.0,189.0,164.0,181.0,148.0,155.0
6887,2022,1469,249.0,244.0,274.0,285.0,260.0,276.0,275.0,240.0,...,235.0,277.0,226.0,248.0,295.0,257.0,230.0,277.0,237.0,251.0
6888,2022,1470,226.0,224.0,245.0,229.0,231.0,239.0,241.0,222.0,...,216.0,209.0,207.0,211.0,225.0,203.0,215.0,231.0,224.0,225.0
6889,2022,1471,252.0,256.0,280.0,267.0,262.0,252.0,281.0,250.0,...,250.0,255.0,261.0,245.0,266.0,240.0,244.0,270.0,256.0,256.0


In [8]:
df_rankings.describe()

,Season,TeamID,BIH,COL,DCI,DOK,DOL,DUN,KPK,KRA,...,PGH,POM,REW,RTH,SAG,STH,WIL,WLK,WOB,WOL
count,6891.000000,6891.000000,6191.000000,6533.000000,4537.000000,5555.000000,6534.000000,5180.000000,4531.000000,4868.000000,...,4878.000000,6535.000000,4885.000000,6191.000000,6538.000000,4538.000000,5550.000000,6535.000000,6184.000000,6191.000000
mean,2012.635031,1284.744594,172.596026,172.537732,175.006833,174.145095,172.564738,173.253282,174.761421,174.414955,...,174.744362,172.590972,174.987718,172.596026,172.670541,175.045394,173.232973,172.590972,172.414457,172.596026
std,5.742555,104.706600,99.507383,99.465787,100.791796,100.321230,99.481689,99.846478,100.698136,100.484912,...,100.644678,99.496679,100.777550,99.507383,99.543157,100.824097,100.162938,99.496679,99.417115,99.507383
min,2003.000000,1101.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,2008.000000,1194.000000,86.500000,86.000000,88.000000,87.000000,86.250000,87.000000,88.000000,87.000000,...,88.000000,86.500000,88.000000,86.500000,87.000000,88.000000,86.000000,86.500000,86.000000,86.500000
50%,2013.000000,1284.000000,172.000000,172.000000,175.000000,174.000000,172.000000,173.000000,175.000000,174.000000,...,175.000000,172.000000,175.000000,172.000000,173.000000,175.000000,173.000000,172.000000,172.000000,172.000000
75%,2018.000000,1376.000000,258.000000,258.000000,262.000000,261.000000,258.000000,259.250000,262.000000,261.000000,...,262.000000,258.000000,262.000000,258.000000,259.000000,262.000000,260.000000,258.000000,258.000000,258.000000
max,2022.000000,1472.000000,358.000000,358.000000,358.000000,358.000000,358.000000,358.000000,358.000000,358.000000,...,358.000000,358.000000,358.000000,358.000000,358.000000,358.000000,358.000000,358.000000,358.000000,358.000000


In [9]:
team0_rankings = df.merge(df_rankings, how='left', 
                          left_on=['season', 'team0'], 
                          right_on=['Season', 'TeamID']).drop(columns=['Season', 'TeamID']).iloc[:, df.shape[1]:]

team1_rankings = df.merge(df_rankings, how='left', 
                          left_on=['season', 'team1'], 
                          right_on=['Season', 'TeamID']).drop(columns=['Season', 'TeamID']).iloc[:, df.shape[1]:]

df[team1_rankings.columns] = team1_rankings - team0_rankings

df

,winner,season,day_number,team0,team1,prev_matchups,common_opps_win_pct_diff,elo_diff,ordinal_elo_diff,BIH,...,PGH,POM,REW,RTH,SAG,STH,WIL,WLK,WOB,WOL
0,0,2003,134,1421,1411,NaN,NaN,-31.631037,17,-13.0,...,NaN,-5.0,NaN,6.0,6.0,NaN,NaN,3.0,19.0,26.0
1,0,2003,136,1112,1436,NaN,-0.333333,-469.244650,133,149.0,...,NaN,141.0,NaN,180.0,156.0,NaN,NaN,151.0,162.0,175.0
2,0,2003,136,1113,1272,NaN,NaN,120.473581,-31,-24.0,...,NaN,-12.0,NaN,-16.0,-8.0,NaN,NaN,-13.0,-21.0,-20.0
3,0,2003,136,1141,1166,NaN,0.000000,117.657793,-22,-21.0,...,NaN,-23.0,NaN,-33.0,-30.0,NaN,NaN,-13.0,-27.0,-32.0
4,0,2003,136,1143,1301,NaN,NaN,-54.718516,22,23.0,...,NaN,8.0,NaN,12.0,6.0,NaN,NaN,18.0,22.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491,1,2022,146,1274,1242,NaN,0.000000,239.449424,-45,-42.0,...,-43.0,-54.0,-39.0,-48.0,-61.0,-46.0,-42.0,-50.0,-45.0,-44.0
2492,1,2022,146,1389,1314,NaN,NaN,212.406858,-88,-102.0,...,-105.0,-88.0,-111.0,-111.0,-114.0,-106.0,-106.0,-83.0,-100.0,-93.0
2493,1,2022,152,1437,1242,NaN,-0.083333,7.104325,-1,-5.0,...,-2.0,-5.0,-1.0,-4.0,-6.0,-7.0,-2.0,-7.0,-3.0,-1.0
2494,1,2022,152,1181,1314,0.5,-0.094203,-85.575138,11,18.0,...,13.0,18.0,16.0,23.0,14.0,19.0,21.0,15.0,19.0,18.0


In [10]:
df.to_pickle('../resources/data/preprocessed/tournament_games.pkl')

'Done'

'Done'